In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
### Go to working directory
%cd drive/Shareddrives/fiver/adib

/content/drive/Shareddrives/fiver/adib


In [ ]:
## If Apex is installed, uninstall it.
!pip uninstall apex

In [4]:
## Install full version of apex (via apex directory)
### Go to apex directory
%cd apex
### Build components
!rm -rf build
### install via setup.py
!python setup.py install --cuda_ext --cpp_ext 

/content/drive/Shareddrives/fiver/adib/apex


torch.__version__  = 1.11.0+cu113


setup.py:121: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
  warnings.warn("Option --pyprof not specified. Not installing PyProf dependencies!")
running install
running bdist_egg
running egg_info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:387: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
adding license file 'LICENSE'
writing manifest file 'apex.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/ape

WARNING! This step is very important. Some functionality from apex is not needed and creates errors if not dealt with. So, before continuing any further, you need to do this step:
1. The ran command that we have just used to install Apex, its output contains a line that starts with 'Installed' then the path of the dependency. (ex. Installed /usr/local/lib/python3.7/dist-packages/apex-0.1-py3.7-linux-x86_64.egg) Go to that path.
2. Inside that directory/path, Go to apex/amp/utils.py (Open it to modify it, not to run it!) Open it with notebook for example.
3. Comment the lines from 95 to 99.
4. Save and exit.

Great ! Now we can move further, by installing some dependencies that are less complex to install than apex. See and run the commands below:

In [5]:
### Go back to working directory
%cd /content/drive/Shareddrives/fiver/adib

/content/drive/Shareddrives/fiver/adib


In [7]:
### Install requirements
!pip install tensorboardX
!pip install yacs
!pip install git+https://github.com/fjchange/opencv_videovision.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/fjchange/opencv_videovision.git to /tmp/pip-req-build-uxti7d0w
  Running command git clone -q https://github.com/fjchange/opencv_videovision.git /tmp/pip-req-build-uxti7d0w
  Created wheel for opencv-videovision: filename=opencv_videovision-0.0.1-py3-none-any.whl size=27368 sha256=87a9ad0aac3b76d9cec86e8b40614f71db11adf46a0e7bf0ecf3c6b0d6c564da
  Stored in directory: /tmp/pip-ephem-wheel-cache-0dmk5_mi/wheels/75/b1/c6/9f3e5fe91d9f2a154df3e67d07581f9a7c4ebf76c1cdedba4a
Successfully built opencv-videovision


In [ ]:
### As a first information to keep in mind, is that the paths of all the files needed (Everything that the model needs like the pretrained model, the datasets the constants used..etc) are present in configs/constant.py . Next steps, we will generate new files that will be used, so, we will also need to change the paths of those files in that constant.py file. But I will walk you through every step in details.

### Before testing our model, there are multiple steps that needs to be done.
### To be able to test our model, we need a folder (step 1) and 3 files (step 2-3-4):
### 1) A Directory containing the videos that we want to test our model on. (Ex. data2/Shoplifting/)
'''
As a first and unique manual step to do, is to select the videos that you want to test our model on, and put them in a directory.
This step is very important as the next steps (and their corresponding scripts to be used) will mainly depend on this step and on that directory.
'''
### 2) A .h5 file that will encapsulates the videos that we want to treat. (Ex. data2/testing/UCFCrime-Frames-testing.h5)
'''
a script 'generate_h5_file_from_directory.py can be directly used. The one step that needs to be done, is to specify the directory. The script can be run by the command below:
" python generate_h5_file_from_directory.py --inputdir <PUT_DIRECTORY_PATH> --outputdir <PUT_WANTED_H5_PATH_NAME.h5>

ex: python generate_h5_file_from_directory.py --inputdir data2/Shoplifting/ --outputdir data2/testing/UCFCrime-Frames-testing.h5

Now, once the new .h5 file is generated, we need to change the path of the test h5 path in configs/constant.py :
1. Copy the outputdir path (you chose that path when running the 'generate_h5_file_from_directory.py' command)
(ex. data2/testing/UCFCrime-Frames-testing.h5)
2. Go to configs/constant.py and open the file to modify it (using notebook or any other coding framework)
3. Search for _C.TEST_H5_PATH='....' and replace the pre-existing path with the copied path.
4. Save and exit

'''
### 3) Temporal annotations: The annotations are provided with the Dataset itself, in order to evaluate the performance of our anomaly detection algorithm. The structure of the file is explained below:
''' 
Each row of 'data/Temporal_Anomaly_Annotation_New.txt' is the annotation for a video, for example:
Shoplifting001_x264.mp4  Shoplifting  1550  2000  -1  -1  
-	The first column is the name of the video
-	The second column is the name of the anomalous event
-	The third column is the starting frame of the event (with 30 frames per second) 
-	The fourth column is the ending frame of the event.
-	For videos in which second  instance of event occurs, fifth and sixth contains starting and ending frames of second instance.  Negative number (-1) means no anomalous event instance. In this example, only one instance of “Shoplifting” is in this video.

This file is already given with the dataset itself and there is no need to generate a new one. (And thus no changes in constant.py file)
'''
### 4) Spatial Annotations: These annotations are used to check how accurate our model is for finding the Shoplifting. As you will see later, our model will output feature maps (as an image) where it highlights the part or zone of the image where the shoplifting occurs. The spatial annotations are matrices that indicates where the event occurs. It helps us especially in the training phase as it shows to our model where to look so that it can understand the event. For testing, it is only used to calculate some evalutation metrics.
'''
This file is already given with the dataset itself and there is no need to generate a new one. (And thus no changes in constant.py file)
'''

Now we are all set, and can run our model on the selected data. For this, you need to run the command below. Note that --vis_UCF argument is used to generate some vizualizations that can be found under foled outputs. These images show how, when, and where the model finds an anomaly (Shoplifting) in the video. These generated images will then be used by another script to generate tagged videos.

In [45]:
!python test.py --gpus 0,1,2,3 --MODEL UCF_C3D --vis_UCF

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
100% 637/637 [03:35<00:00,  2.95it/s]
UCF_C3D: AUC 78.06%, FAR 0.00%, GAP 8.82%


Now, only one step remains, which is to generate tagged videos. 
This is done using the command below: (Make sure to check to paths, since this notebook was created using Google Colab, the path need to be adjusted for your case. The same goes to all paths that are present in configs/constant.py

In [65]:
!python utils/generate_tagged_videos.py --inputdir /content/drive/Shareddrives/fiver/adib/data2/testing/ --featuremaps /content/drive/Shareddrives/fiver/adib/outputs/ --tagged_videos_dir /content/drive/Shareddrives/fiver/adib/data2/testing/tagged/

/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting004_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting001_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting005_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting007_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting010_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting015_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting016_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting017_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting020_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting021_x264tagged.mp4
Done!
/content/drive/Shareddrives/fiver/adib/data2/testing/Shoplifting033_x264tagged.mp4
Done!
/content/drive/Shared